En este ejemplo se **viola el principio de Protected Variations** al exponer directamente dependencias críticas y al no encapsular cambios en abstracciones. El código tiene una **dificultad mayor**, ya que maneja múltiples clases y estructuras más complejas relacionadas con pagos en una tienda en línea.

---

### **Descripción del problema**
- Tenemos un **sistema de pagos** con distintos métodos de pago (`PayPal`, `CreditCard` y `BankTransfer`).
- La clase `PaymentProcessor` **depende directamente** de las implementaciones concretas de los métodos de pago.
- Si queremos agregar un nuevo método de pago, **tenemos que modificar el código existente**, lo que rompe el principio de **Protected Variations**.
- **No hay una interfaz abstracta** para proteger `PaymentProcessor` de cambios en las clases concretas de pago.
- Se mezcla lógica de pago con la lógica de órdenes.

---

### **Código que Viola Protected Variations**
```python
from typing import Dict

class PayPal:
    """Clase que maneja pagos con PayPal."""
    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con PayPal...")
        return True  # Simulación de pago exitoso

class CreditCard:
    """Clase que maneja pagos con tarjeta de crédito."""
    def process_payment(self, amount: float, card_number: str) -> bool:
        print(f"Procesando pago de ${amount:.2f} con tarjeta {card_number}...")
        return True  # Simulación de pago exitoso

class BankTransfer:
    """Clase que maneja pagos con transferencia bancaria."""
    def process_payment(self, amount: float, account_number: str) -> bool:
        print(f"Procesando pago de ${amount:.2f} con cuenta bancaria {account_number}...")
        return True  # Simulación de pago exitoso

class Order:
    """Clase que maneja órdenes y pagos directamente con clases concretas."""
    def __init__(self, order_id: str, total: float) -> None:
        self.order_id = order_id
        self.total = total

    def process_order(self, payment_method: str, details: Dict[str, str]) -> None:
        """Procesa el pedido dependiendo del método de pago."""
        if payment_method == "paypal":
            paypal = PayPal()
            success = paypal.process_payment(self.total)
        elif payment_method == "creditcard":
            credit_card = CreditCard()
            success = credit_card.process_payment(self.total, details["card_number"])
        elif payment_method == "banktransfer":
            bank_transfer = BankTransfer()
            success = bank_transfer.process_payment(self.total, details["account_number"])
        else:
            print("Método de pago no soportado.")
            return
        
        if success:
            print(f"Orden {self.order_id} procesada exitosamente.")
        else:
            print(f"Orden {self.order_id} falló en el pago.")

# Uso del código
order = Order("ORD123", 150.75)
order.process_order("creditcard", {"card_number": "1234-5678-9876-5432"})
```

---

### **¿Por qué este código viola Protected Variations?**
1. **El `Order` está directamente acoplado a clases concretas (`PayPal`, `CreditCard`, `BankTransfer`)**:
   - Si se agrega un nuevo método de pago, hay que modificar `process_order`, lo que rompe el principio de protección contra cambios.
   
2. **No hay abstracciones o interfaces**:
   - No hay una interfaz común para los métodos de pago, lo que obliga a `Order` a manejar cada uno de manera diferente.
   
3. **Dependencias innecesarias dentro de `Order`**:
   - `Order` tiene que saber cómo instanciar y usar cada método de pago en lugar de depender de una interfaz genérica.

---

## **Versión Refactorizada con Protected Variations**
Ahora, refactorizamos el código utilizando **una abstracción para protegerse de cambios futuros**.

```python
from typing import Protocol, Dict

class PaymentMethod(Protocol):
    """Interfaz para proteger el sistema de variaciones en métodos de pago."""
    def process_payment(self, amount: float) -> bool:
        ...

class PayPal(PaymentMethod):
    """Clase que maneja pagos con PayPal."""
    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con PayPal...")
        return True

class CreditCard(PaymentMethod):
    """Clase que maneja pagos con tarjeta de crédito."""
    def __init__(self, card_number: str) -> None:
        self.card_number = card_number

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con tarjeta {self.card_number}...")
        return True

class BankTransfer(PaymentMethod):
    """Clase que maneja pagos con transferencia bancaria."""
    def __init__(self, account_number: str) -> None:
        self.account_number = account_number

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con cuenta bancaria {self.account_number}...")
        return True

class Order:
    """Clase que gestiona órdenes sin acoplarse a implementaciones concretas."""
    def __init__(self, order_id: str, total: float, payment_method: PaymentMethod) -> None:
        self.order_id = order_id
        self.total = total
        self.payment_method = payment_method

    def process_order(self) -> None:
        """Procesa la orden usando un método de pago genérico."""
        success = self.payment_method.process_payment(self.total)
        
        if success:
            print(f"Orden {self.order_id} procesada exitosamente.")
        else:
            print(f"Orden {self.order_id} falló en el pago.")

# Uso del código refactorizado
payment_method = CreditCard("1234-5678-9876-5432")
order = Order("ORD123", 150.75, payment_method)
order.process_order()
```

---

### **¿Por qué esta versión respeta Protected Variations?**
✅ **Encapsula las variaciones con una interfaz (`PaymentMethod`)**  
   - Ahora `Order` solo depende de `PaymentMethod`, sin importar si es PayPal, tarjeta de crédito o transferencia bancaria.

✅ **Permite agregar nuevos métodos de pago sin modificar `Order`**  
   - Si queremos agregar `BitcoinPayment`, simplemente implementamos `PaymentMethod`, sin modificar `Order`.

✅ **Reduce el acoplamiento**  
   - `Order` no necesita conocer los detalles de cada método de pago, solo usa la interfaz.

---

### **Beneficios adicionales**
- **Extensibilidad**: Agregar nuevos métodos de pago es más fácil.
- **Menos errores al modificar código**: No hay que modificar `Order` cada vez que agregamos un nuevo método.
- **Mejor testabilidad**: Podemos **simular un método de pago falso** en pruebas unitarias sin modificar `Order`.

---

### **Conclusión**
El primer código **violaba el principio de Protected Variations** porque `Order` estaba **directamente acoplado** a las implementaciones concretas de pago, lo que hacía difícil modificar o extender el código sin cambiar varias partes del sistema.  
En la versión refactorizada, se usó una **abstracción (`PaymentMethod`)** para desacoplar `Order` de los métodos de pago, protegiendo el sistema contra variaciones y facilitando la extensión futura.


In [2]:
from typing import Dict

class PayPal:
    """Clase que maneja pagos con PayPal."""
    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con PayPal...")
        return True  # Simulación de pago exitoso

class CreditCard:
    """Clase que maneja pagos con tarjeta de crédito."""
    def process_payment(self, amount: float, card_number: str) -> bool:
        print(f"Procesando pago de ${amount:.2f} con tarjeta {card_number}...")
        return True  # Simulación de pago exitoso

class BankTransfer:
    """Clase que maneja pagos con transferencia bancaria."""
    def process_payment(self, amount: float, account_number: str) -> bool:
        print(f"Procesando pago de ${amount:.2f} con cuenta bancaria {account_number}...")
        return True  # Simulación de pago exitoso

class Order:
    """Clase que maneja órdenes y pagos directamente con clases concretas."""
    def __init__(self, order_id: str, total: float) -> None:
        self.order_id = order_id
        self.total = total

    def process_order(self, payment_method: str, details: Dict[str, str]) -> None:
        """Procesa el pedido dependiendo del método de pago."""
        if payment_method == "paypal":
            paypal = PayPal()
            success = paypal.process_payment(self.total)
        elif payment_method == "creditcard":
            credit_card = CreditCard()
            success = credit_card.process_payment(self.total, details["card_number"])
        elif payment_method == "banktransfer":
            bank_transfer = BankTransfer()
            success = bank_transfer.process_payment(self.total, details["account_number"])
        else:
            print("Método de pago no soportado.")
            return
        
        if success:
            print(f"Orden {self.order_id} procesada exitosamente.")
        else:
            print(f"Orden {self.order_id} falló en el pago.")

# Uso del código
order = Order("ORD123", 150.75)
order.process_order("creditcard", {"card_number": "1234-5678-9876-5432"})


Procesando pago de $150.75 con tarjeta 1234-5678-9876-5432...
Orden ORD123 procesada exitosamente.


In [1]:
from typing import Protocol, Dict

class PaymentMethod(Protocol):
    """Interfaz para proteger el sistema de variaciones en métodos de pago."""
    def process_payment(self, amount: float) -> bool:
        ...

class PayPal(PaymentMethod):
    """Clase que maneja pagos con PayPal."""
    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con PayPal...")
        return True

class CreditCard(PaymentMethod):
    """Clase que maneja pagos con tarjeta de crédito."""
    def __init__(self, card_number: str) -> None:
        self.card_number = card_number

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con tarjeta {self.card_number}...")
        return True

class BankTransfer(PaymentMethod):
    """Clase que maneja pagos con transferencia bancaria."""
    def __init__(self, account_number: str) -> None:
        self.account_number = account_number

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con cuenta bancaria {self.account_number}...")
        return True

class Order:
    """Clase que gestiona órdenes sin acoplarse a implementaciones concretas."""
    def __init__(self, order_id: str, total: float, payment_method: PaymentMethod) -> None:
        self.order_id = order_id
        self.total = total
        self.payment_method = payment_method

    def process_order(self) -> None:
        """Procesa la orden usando un método de pago genérico."""
        success = self.payment_method.process_payment(self.total)
        
        if success:
            print(f"Orden {self.order_id} procesada exitosamente.")
        else:
            print(f"Orden {self.order_id} falló en el pago.")

# Uso del código refactorizado
payment_method = CreditCard("1234-5678-9876-5432")
order = Order("ORD123", 150.75, payment_method)
order.process_order()


Procesando pago de $150.75 con tarjeta 1234-5678-9876-5432...
Orden ORD123 procesada exitosamente.


# Hay lugar para mejoras?

Sí, aunque la refactorización del código para respetar el principio de **Protected Variations** fue un gran avance, **aún hay áreas de mejora**. Vamos a analizar los aspectos que pueden optimizarse y proponer una versión aún más robusta.

---

## **¿Qué problemas persisten en la versión refactorizada?**
1. **El `OrderProcessor` sigue dependiendo de `PaymentMethod`**  
   - Aunque `OrderProcessor` ya no está acoplado a clases concretas (`PayPal`, `CreditCard`, `BankTransfer`), aún **depende de la interfaz `PaymentMethod`**.
   - Si se agregan nuevas reglas de validación o lógica adicional de pagos, **podría ser necesario modificar `OrderProcessor`**, lo que todavía genera **cierto nivel de acoplamiento**.

2. **Falta de una Fábrica para gestionar la creación de métodos de pago**  
   - Actualmente, `OrderProcessor` **recibe un método de pago en el constructor**, lo que obliga a la lógica externa a instanciar el método de pago antes de procesar el pedido.
   - Esto hace que **el código del cliente tenga que conocer los métodos de pago concretos**, lo cual es una **violación del principio de Protected Variations**.

3. **No hay validaciones de pago según el método seleccionado**  
   - Cada método de pago puede requerir información específica (ej. tarjeta de crédito necesita un número de tarjeta, transferencia bancaria necesita una cuenta).
   - Actualmente, `OrderProcessor` **no maneja validaciones**, lo que puede provocar errores en tiempo de ejecución.

---

## **Solución: Introducir un Factory y un Validator**
Para solucionar estos problemas:
✅ **Se introduce un `PaymentFactory`** para que `OrderProcessor` **no tenga que conocer los métodos de pago** y pueda delegar su creación.  
✅ **Se introduce un `PaymentValidator`** para verificar si los datos del pago son correctos antes de procesarlo.  
✅ **`OrderProcessor` ahora solo conoce `PaymentProcessorInterface`**, lo que elimina cualquier acoplamiento con métodos de pago específicos.  

---

### **Código Mejorado**
```python
from typing import Protocol, Dict, Optional

class PaymentProcessorInterface(Protocol):
    """Interfaz para definir un método de pago genérico."""
    def process_payment(self, amount: float) -> bool:
        ...

class PayPalPayment(PaymentProcessorInterface):
    """Clase que maneja pagos con PayPal."""
    def __init__(self, email: str) -> None:
        self.email = email

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con PayPal para {self.email}")
        return True

class CreditCardPayment(PaymentProcessorInterface):
    """Clase que maneja pagos con tarjeta de crédito."""
    def __init__(self, card_number: str) -> None:
        self.card_number = card_number

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con tarjeta {self.card_number}")
        return True

class BankTransferPayment(PaymentProcessorInterface):
    """Clase que maneja pagos con transferencia bancaria."""
    def __init__(self, account_number: str) -> None:
        self.account_number = account_number

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con cuenta bancaria {self.account_number}")
        return True

class PaymentValidator:
    """Valida los datos de pago antes de procesarlos."""
    @staticmethod
    def validate(payment_method: str, details: Dict[str, str]) -> bool:
        if payment_method == "paypal":
            return "email" in details
        elif payment_method == "creditcard":
            return "card_number" in details
        elif payment_method == "banktransfer":
            return "account_number" in details
        return False

class PaymentFactory:
    """Fábrica que crea instancias de métodos de pago sin acoplar `OrderProcessor` a clases concretas."""
    @staticmethod
    def create_payment_method(payment_method: str, details: Dict[str, str]) -> Optional[PaymentProcessorInterface]:
        if not PaymentValidator.validate(payment_method, details):
            print(f"Error: Datos inválidos para el método de pago '{payment_method}'")
            return None

        if payment_method == "paypal":
            return PayPalPayment(details["email"])
        elif payment_method == "creditcard":
            return CreditCardPayment(details["card_number"])
        elif payment_method == "banktransfer":
            return BankTransferPayment(details["account_number"])
        return None

class OrderProcessor:
    """Procesa órdenes sin depender de clases concretas de pago."""
    def __init__(self, order_id: str, total: float) -> None:
        self.order_id = order_id
        self.total = total

    def process_order(self, payment_method: str, details: Dict[str, str]) -> None:
        """Procesa la orden usando un método de pago generado por la fábrica."""
        payment_processor = PaymentFactory.create_payment_method(payment_method, details)

        if not payment_processor:
            print(f"⚠️ No se pudo procesar la orden {self.order_id} debido a un método de pago inválido.")
            return

        if payment_processor.process_payment(self.total):
            print(f"✅ Orden {self.order_id} procesada exitosamente.")
        else:
            print(f"❌ Falló el pago para la orden {self.order_id}.")

# Uso del código mejorado
order1 = OrderProcessor("ORD123", 150.75)
order1.process_order("creditcard", {"card_number": "1234-5678-9876-5432"})

order2 = OrderProcessor("ORD124", 89.99)
order2.process_order("paypal", {"email": "user@example.com"})

order3 = OrderProcessor("ORD125", 200.00)
order3.process_order("banktransfer", {"account_number": "987654321"})
```

---

## **¿Por qué esta versión es aún mejor?**
✅ **`OrderProcessor` ya no conoce ninguna clase concreta de pago**  
   - Antes, `OrderProcessor` tenía que conocer `PayPalPayment`, `CreditCardPayment` y `BankTransferPayment`.  
   - Ahora, simplemente llama a `PaymentFactory.create_payment_method()` y **se protege contra cambios en los métodos de pago**.  

✅ **Se ha centralizado la validación con `PaymentValidator`**  
   - Antes, `OrderProcessor` no verificaba si los datos eran correctos antes de procesar el pago.  
   - Ahora, `PaymentValidator` asegura que el método de pago tenga los datos adecuados.  

✅ **Se introduce el `PaymentFactory` como capa de abstracción**  
   - `OrderProcessor` no necesita instanciar manualmente un método de pago.  
   - Si se agrega un nuevo método de pago (`CryptoPayment`, `ApplePayPayment`, etc.), **no hay que modificar `OrderProcessor`**, solo `PaymentFactory`.  

✅ **El diseño cumple con Open/Closed Principle (OCP)**  
   - **Se pueden agregar nuevos métodos de pago sin modificar el código existente**.  
   - `OrderProcessor` no tiene que cambiar, solo se actualiza `PaymentFactory`.  

✅ **Mayor separación de responsabilidades**  
   - `OrderProcessor` **se encarga solo de las órdenes**.  
   - `PaymentFactory` **se encarga de instanciar métodos de pago**.  
   - `PaymentValidator` **se encarga de validar los datos de pago**.  

---

## **Comparación Antes vs. Después**
| Problema | Código Anterior 🚨 | Código Mejorado ✅ |
|----------|-------------------|-------------------|
| **Acoplamiento** | `OrderProcessor` conocía todas las clases de pago. | `OrderProcessor` usa `PaymentFactory`, sin conocer clases concretas. |
| **Extensibilidad** | Agregar un nuevo método de pago requería modificar `OrderProcessor`. | Se pueden agregar nuevos métodos sin modificar `OrderProcessor`. |
| **Validación de datos** | No validaba los datos antes del pago. | `PaymentValidator` verifica si los datos son correctos antes de procesar el pago. |

---

## **Conclusión**
La mejora final refuerza **Protected Variations** al **aislar completamente a `OrderProcessor` de cualquier variación en los métodos de pago**.  
Con **una fábrica (`PaymentFactory`) y un validador (`PaymentValidator`)**, el sistema ahora es **totalmente modular, extensible y protegido contra cambios**. 🚀

In [3]:
from typing import Protocol, Dict, Optional

class PaymentProcessorInterface(Protocol):
    """Interfaz para definir un método de pago genérico."""
    def process_payment(self, amount: float) -> bool:
        ...

class PayPalPayment(PaymentProcessorInterface):
    """Clase que maneja pagos con PayPal."""
    def __init__(self, email: str) -> None:
        self.email = email

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con PayPal para {self.email}")
        return True

class CreditCardPayment(PaymentProcessorInterface):
    """Clase que maneja pagos con tarjeta de crédito."""
    def __init__(self, card_number: str) -> None:
        self.card_number = card_number

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con tarjeta {self.card_number}")
        return True

class BankTransferPayment(PaymentProcessorInterface):
    """Clase que maneja pagos con transferencia bancaria."""
    def __init__(self, account_number: str) -> None:
        self.account_number = account_number

    def process_payment(self, amount: float) -> bool:
        print(f"Procesando pago de ${amount:.2f} con cuenta bancaria {self.account_number}")
        return True

class PaymentValidator:
    """Valida los datos de pago antes de procesarlos."""
    @staticmethod
    def validate(payment_method: str, details: Dict[str, str]) -> bool:
        if payment_method == "paypal":
            return "email" in details
        elif payment_method == "creditcard":
            return "card_number" in details
        elif payment_method == "banktransfer":
            return "account_number" in details
        return False

class PaymentFactory:
    """Fábrica que crea instancias de métodos de pago sin acoplar `OrderProcessor` a clases concretas."""
    @staticmethod
    def create_payment_method(payment_method: str, details: Dict[str, str]) -> Optional[PaymentProcessorInterface]:
        if not PaymentValidator.validate(payment_method, details):
            print(f"Error: Datos inválidos para el método de pago '{payment_method}'")
            return None

        if payment_method == "paypal":
            return PayPalPayment(details["email"])
        elif payment_method == "creditcard":
            return CreditCardPayment(details["card_number"])
        elif payment_method == "banktransfer":
            return BankTransferPayment(details["account_number"])
        return None

class OrderProcessor:
    """Procesa órdenes sin depender de clases concretas de pago."""
    def __init__(self, order_id: str, total: float) -> None:
        self.order_id = order_id
        self.total = total

    def process_order(self, payment_method: str, details: Dict[str, str]) -> None:
        """Procesa la orden usando un método de pago generado por la fábrica."""
        payment_processor = PaymentFactory.create_payment_method(payment_method, details)

        if not payment_processor:
            print(f"⚠️ No se pudo procesar la orden {self.order_id} debido a un método de pago inválido.")
            return

        if payment_processor.process_payment(self.total):
            print(f"✅ Orden {self.order_id} procesada exitosamente.")
        else:
            print(f"❌ Falló el pago para la orden {self.order_id}.")

# Uso del código mejorado
order1 = OrderProcessor("ORD123", 150.75)
order1.process_order("creditcard", {"card_number": "1234-5678-9876-5432"})

order2 = OrderProcessor("ORD124", 89.99)
order2.process_order("paypal", {"email": "user@example.com"})

order3 = OrderProcessor("ORD125", 200.00)
order3.process_order("banktransfer", {"account_number": "987654321"})


Procesando pago de $150.75 con tarjeta 1234-5678-9876-5432
✅ Orden ORD123 procesada exitosamente.
Procesando pago de $89.99 con PayPal para user@example.com
✅ Orden ORD124 procesada exitosamente.
Procesando pago de $200.00 con cuenta bancaria 987654321
✅ Orden ORD125 procesada exitosamente.
